In [2]:
from IPython.display import clear_output, display
from time import sleep

In [3]:
import numpy as np
from itertools import product

import softmax_helper
from importlib import reload
reload(softmax_helper)

<module 'softmax_helper' from '/Users/rahul/python_projects/gqp-takeda/notebooks/softmax_helper.py'>

In [4]:
import pandas as pd

label_info = pd.read_csv('../data/phase1/activity_labels.txt', delim_whitespace=True, header=None, index_col=False)
label_mapping = dict()

for index, row_data in label_info.iterrows():
    label_mapping.update({row_data[0]-1 : row_data[1]})

In [16]:
X_train = np.load('../data/phase1/train/X_train.npy')
Y_train = np.load('../data/phase1/train/Y_train.npy')
X_valid = np.load('../data/phase1/train/X_valid.npy')
Y_valid = np.load('../data/phase1/train/Y_valid.npy')
X_test  = np.load('../data/phase1/train/X_test.npy')
Y_test  = np.load('../data/phase1/train/Y_test.npy')

In [17]:
X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape, X_test.shape, Y_test.shape

((6213, 86), (6213, 12), (1554, 86), (1554, 12), (3162, 561), (3162, 12))

In [18]:
alpha = 1e-4; mini_batch_size = 128; learning_rate = 3e-2; epochs = 1000

In [19]:
w = softmax_helper.stochastic_gradient_descent(X_train, Y_train, alpha, mini_batch_size, learning_rate, epochs)

Current loss: 1.0295878303478663; epoch: 0; accuracy: 0.6933333333333334
Current loss: 0.3649937484959556; epoch: 20; accuracy: 0.8066666666666666
Current loss: 0.24511965262405522; epoch: 40; accuracy: 0.88
Current loss: 0.2370731520933753; epoch: 60; accuracy: 0.8733333333333333
Current loss: 0.2508971059585292; epoch: 80; accuracy: 0.8466666666666667
Current loss: 0.22914589533983382; epoch: 100; accuracy: 0.8533333333333334
Current loss: 0.18669154477366964; epoch: 120; accuracy: 0.8733333333333333
Current loss: 0.19631089816919672; epoch: 140; accuracy: 0.8733333333333333
Current loss: 0.1704051044922564; epoch: 160; accuracy: 0.9133333333333333
Current loss: 0.1574273379501263; epoch: 180; accuracy: 0.9
Current loss: 0.1482948844748124; epoch: 200; accuracy: 0.9133333333333333
Current loss: 0.14066152895583106; epoch: 220; accuracy: 0.9266666666666666
Current loss: 0.201851698391325; epoch: 240; accuracy: 0.84
Current loss: 0.15468262674311067; epoch: 260; accuracy: 0.89333333333

In [20]:
best_w = w

In [21]:
train_probs = softmax_helper.get_probs(X_train, best_w)
train_loss = softmax_helper.cross_entropy_loss(Y_train, train_probs)
train_accuracy = softmax_helper.get_accuracy(Y_train, train_probs)
print("Final training loss: {}".format(train_loss))
print("Final training accuracy: {}".format(train_accuracy))

Final training loss: 0.11293542827395964
Final training accuracy: 0.9332045710606792


In [22]:
valid_probs = softmax_helper.get_probs(X_valid, best_w)
valid_loss = softmax_helper.cross_entropy_loss(Y_valid, valid_probs)
valid_accuracy = softmax_helper.get_accuracy(Y_valid, valid_probs)
print("Validation loss: {}".format(valid_loss))
print("Validation accuracy: {}".format(valid_accuracy))

Validation loss: 0.13046773752750365
Validation accuracy: 0.9111969111969112


## Evaluation

In [23]:
from sklearn.metrics import classification_report, classification

### Training:

In [24]:
print(
    classification_report(
        y_true=Y_train.argmax(axis=1),
        y_pred=train_probs.argmax(axis=1),
        labels=list(label_mapping.keys()),
        target_names=list(label_mapping.values()),
    )
)

                    precision    recall  f1-score   support

           WALKING       0.97      0.97      0.97       971
  WALKING_UPSTAIRS       0.97      0.98      0.97       869
WALKING_DOWNSTAIRS       0.96      0.98      0.97       783
           SITTING       0.88      0.86      0.87      1030
          STANDING       0.90      0.91      0.90      1143
            LAYING       0.95      0.94      0.94      1129
      STAND_TO_SIT       0.91      0.79      0.85        38
      SIT_TO_STAND       0.89      0.89      0.89        19
        SIT_TO_LIE       0.86      0.86      0.86        57
        LIE_TO_SIT       0.92      0.85      0.88        53
      STAND_TO_LIE       0.91      0.85      0.88        73
      LIE_TO_STAND       0.85      0.85      0.85        48

         micro avg       0.93      0.93      0.93      6213
         macro avg       0.91      0.90      0.90      6213
      weighted avg       0.93      0.93      0.93      6213



### Validation:

In [25]:
print(
    classification_report(
        y_true=Y_valid.argmax(axis=1),
        y_pred=valid_probs.argmax(axis=1),
        labels=list(label_mapping.keys()),
        target_names=list(label_mapping.values())
    )
)

                    precision    recall  f1-score   support

           WALKING       0.97      0.95      0.96       255
  WALKING_UPSTAIRS       0.93      0.97      0.95       204
WALKING_DOWNSTAIRS       0.96      0.97      0.96       204
           SITTING       0.84      0.84      0.84       263
          STANDING       0.87      0.88      0.87       280
            LAYING       0.95      0.93      0.94       284
      STAND_TO_SIT       0.75      0.67      0.71         9
      SIT_TO_STAND       1.00      0.75      0.86         4
        SIT_TO_LIE       1.00      0.61      0.76        18
        LIE_TO_SIT       0.71      0.71      0.71         7
      STAND_TO_LIE       0.72      0.76      0.74        17
      LIE_TO_STAND       0.62      0.56      0.59         9

         micro avg       0.91      0.91      0.91      1554
         macro avg       0.86      0.80      0.82      1554
      weighted avg       0.91      0.91      0.91      1554



## Hyperparam grid

In [ ]:
all_epochs = [10, 50, 100, 200]
all_batch_sizes = [100, 200, 500, 1000]
all_learning_rates = [0.0001, 0.001, 0.01, 0.1]
all_alphas = [0.0001, 0.01, 0.1]

min_loss = np.inf
best_params = (0,0,0,0)

all_combinations = list(product(all_epochs, all_batch_sizes, all_alphas, all_learning_rates,))

print("# hyper param comb: {}".format(len(all_combinations)))
input("Press Enter to continue...")

for param in all_combinations:
    try:
        epochs, mini_batch_size, alpha, learning_rate = param

        print("epochs: {}; mini_batch_size: {}; alpha: {}; learning_rate: {}".format(*param))

        w = softmax_helper.stochastic_gradient_descent(X_train, Y_train, alpha, mini_batch_size, learning_rate, epochs)

        valid_probs = softmax_helper.get_probs(X_valid, w)
        valid_loss = softmax_helper.cross_entropy_loss(Y_valid, valid_probs)
        valid_accuracy = softmax_helper.get_accuracy(Y_valid, valid_probs)
        print("Valid loss: {}".format(valid_loss))
        print("Valid accuracy: {}".format(valid_accuracy))

        if valid_loss < min_loss:
            min_loss = valid_loss
            best_params = param
    except KeyboardInterrupt as e:
        break

epochs, mini_batch_size, alpha, learning_rate = best_params
finalTrainData = np.append(X_train, X_valid, axis=0)
finalTrainLabels = np.append(Y_train, Y_valid, axis=0)

best_w = softmax_helper.stochastic_gradient_descent(finalTrainData, finalTrainLabels, alpha, mini_batch_size, learning_rate, epochs)

In [ ]:

train_probs = softmax_helper.get_probs(X_train, best_w)
train_loss = softmax_helper.cross_entropy_loss(Y_train, train_probs)
train_accuracy = softmax_helper.get_accuracy(Y_train, train_probs)
print("Final training loss: {}".format(train_loss))
print("Final training accuracy: {}".format(train_accuracy))

valid_probs = softmax_helper.get_probs(X_valid, best_w)
valid_loss = softmax_helper.cross_entropy_loss(Y_valid, valid_probs)
valid_accuracy = softmax_helper.get_accuracy(Y_valid, valid_probs)
print("Validation loss: {}".format(valid_loss))
print("Validation accuracy: {}".format(valid_accuracy))
